In [1]:
import pandas as pd
import nltk
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string
# from langdetect import detect
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

Traemos todos los datasets de reviews

In [3]:
df1 = pd.read_csv(r"data\reviews_0-250.csv")
df2 = pd.read_csv(r"data\reviews_250-500.csv")
df3 = pd.read_csv(r"data\reviews_500-750.csv")
df4 = pd.read_csv(r"data\reviews_750-1250.csv")
df5 = pd.read_csv(r"data\reviews_1250-end.csv")

C:\Users\marti\AppData\Local\Temp\ipykernel_10576\2264000620.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r"data\reviews_0-250.csv")
C:\Users\marti\AppData\Local\Temp\ipykernel_10576\2264000620.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv(r"data\reviews_750-1250.csv")
C:\Users\marti\AppData\Local\Temp\ipykernel_10576\2264000620.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df5 = pd.read_csv(r"data\reviews_1250-end.csv")


In [4]:
df = pd.concat([df1,df2,df3, df4, df5])

In [5]:
len(df)

1094411

Tenemos más de 1 millón de reviews de productos

In [6]:
df.columns

Index(['Unnamed: 0', 'author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')

In [7]:
df[["product_id", "review_text", "rating"]].head()

,product_id,review_text,rating
0,P504322,I use this with the Nudestix “Citrus Clean Bal...,5
1,P420652,I bought this lip mask after reading the revie...,1
2,P420652,My review title says it all! I get so excited ...,5
3,P420652,I’ve always loved this formula for a long time...,5
4,P420652,"If you have dry cracked lips, this is a must h...",5


Las agrupamos por producto

In [8]:
df['review_text'] = df['review_text'].astype(str)

# Agrupar por 'product_id', concatenar las 'review_text' y calcular el promedio de 'rating'
grouped_df = df.groupby('product_id').agg({
    'review_text': lambda x: ' '.join(x),
    'rating': 'mean', 
    "author_id": "count"
}).reset_index()

# Renombrar las columnas para mayor claridad
grouped_df.columns = ['product_id', 'combined_reviews', 'average_rating', "review_count"]
grouped_df

,product_id,combined_reviews,average_rating,review_count
0,P107306,"Nice consistency, however I’m not sure it’s do...",4.031621,253
1,P114902,I wanted to try these to help with the texture...,4.419882,1529
2,P12045,"I have sensitive, red skin, and I added this t...",4.443391,1687
3,P122651,My absolute favourite toner! My skin is a comb...,4.515000,200
4,P122661,I love that this isn’t too abrasive and it doe...,4.532099,810
...,...,...,...,...
2346,P94421,nan Absolutely love the immediate results you ...,4.244681,2914
2347,P94812,I used this once only. Made my skin feel dry a...,4.613767,523
2348,P9939,Murad has always been a go-to Product lines th...,4.308824,272
2349,P9940,This makes my skin very greasy and its very ir...,4.248492,829


Tenemos reviews sobre 2351 productos

In [9]:
df = grouped_df.copy()

Procesamiento

In [10]:
stopwords_en = stopwords.words('english')
punctuation = string.punctuation

import re

def normalizar_texto(texto):
    # Tokenización:
    tokens = nltk.word_tokenize(str(texto))
    # Eliminar mayúsculas:
    tokens = [token.lower() for token in tokens]
    # Eliminar stop words y puntuación:
    tokens = [token for token in tokens if token not in stopwords_en and token not in punctuation]
    # Eliminar números:
    tokens = [re.sub(r'\d+', '', token) for token in tokens]
    # Convertir lista de tokens a cadena de texto:
    texto_normalizado = ' '.join(tokens)
    return texto_normalizado

In [11]:
df["texto_limpio"] = df.apply(lambda row: normalizar_texto(row["combined_reviews"]), axis = 1)
df.head()

,product_id,combined_reviews,average_rating,review_count,texto_limpio
0,P107306,"Nice consistency, however I’m not sure it’s do...",4.031621,253,nice consistency however ’ sure ’ ’ promised ’...
1,P114902,I wanted to try these to help with the texture...,4.419882,1529,wanted try help texture skin well visible pore...
2,P12045,"I have sensitive, red skin, and I added this t...",4.443391,1687,sensitive red skin added skin routine morning ...
3,P122651,My absolute favourite toner! My skin is a comb...,4.515000,200,absolute favourite toner skin combination supe...
4,P122661,I love that this isn’t too abrasive and it doe...,4.532099,810,love ’ abrasive leave skin feeling soft also l...


Lematización

In [13]:
# Crear un objeto lematizador
lemmatizer = WordNetLemmatizer()

# Definir una función para lematizar un texto
def lemmatize_text(text):
    tokens = word_tokenize(text)  # Tokenizar el texto en palabras
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lematizar cada palabra
    return ' '.join(lemmatized_tokens)  # Unir las palabras lematizadas en un solo texto

# Aplicar la lematización a la columna "texto_limpio"
df['texto_lemmatizado'] = df['texto_limpio'].apply(lemmatize_text)

In [14]:
df.head()

,product_id,combined_reviews,average_rating,review_count,texto_limpio,texto_lemmatizado
0,P107306,"Nice consistency, however I’m not sure it’s do...",4.031621,253,nice consistency however ’ sure ’ ’ promised ’...,nice consistency however ’ sure ’ ’ promised ’...
1,P114902,I wanted to try these to help with the texture...,4.419882,1529,wanted try help texture skin well visible pore...,wanted try help texture skin well visible pore...
2,P12045,"I have sensitive, red skin, and I added this t...",4.443391,1687,sensitive red skin added skin routine morning ...,sensitive red skin added skin routine morning ...
3,P122651,My absolute favourite toner! My skin is a comb...,4.515000,200,absolute favourite toner skin combination supe...,absolute favourite toner skin combination supe...
4,P122661,I love that this isn’t too abrasive and it doe...,4.532099,810,love ’ abrasive leave skin feeling soft also l...,love ’ abrasive leave skin feeling soft also l...


In [15]:
vectorizer = TfidfVectorizer(min_df=15, max_df = 0.8, analyzer='word', token_pattern=r'\w+',
                             use_idf=True, smooth_idf=True, sublinear_tf=True,
                             stop_words='english', ngram_range=(1, 2))
vectors = vectorizer.fit_transform(df['texto_lemmatizado'])
vectors.shape

(2351, 187519)

In [16]:
corpus_tfidf_array = vectors.toarray()

In [17]:
# Convertir la matriz TF-IDF en un DataFrame
tfidf_df = pd.DataFrame(corpus_tfidf_array, columns=vectorizer.vocabulary_.keys())

In [18]:
tfidf_df.shape

(2351, 187519)

In [19]:
# Calcular la suma de los valores TF-IDF para cada columna
column_sum = tfidf_df.sum()

# Ordenar las columnas por su suma de valores TF-IDF en orden descendente
columnas_importantes = column_sum.sort_values(ascending=False).index

# Especificar el número de columnas más importantes que deseas mantener
num_columnas = 2000  # Actualiza este valor según tus necesidades

# Seleccionar las primeras n columnas más importantes del DataFrame tfidf_df
tfidf_importantes = tfidf_df[columnas_importantes[:num_columnas]]

# Mostrar el DataFrame resultante con las columnas más importantes
tfidf_importantes.head()

,insensitive,skin look,surprised stuff,great warm,told best,happy new,worked completely,revamp,area absorbs,fiance,...,add lovely,accordingly,murad really,spf usually,irritation drying,tried otc,allows layer,helped large,line worth,lightweight effective
0,0.010536,0.009554,0.007561,0.031998,0.007460,0.013716,0.022896,0.010454,0.017137,0.009226,...,0.000000,0.000000,0.007670,0.000000,0.008947,0.007368,0.0,0.007507,0.000000,0.007661
1,0.006762,0.012645,0.008969,0.008194,0.008876,0.000000,0.008317,0.007315,0.005093,0.007109,...,0.003368,0.002700,0.002690,0.002554,0.000000,0.008261,0.0,0.006869,0.004738,0.005638
2,0.010190,0.009277,0.010387,0.008186,0.010309,0.010279,0.007277,0.010296,0.011717,0.010027,...,0.000000,0.011777,0.005266,0.004999,0.000000,0.006270,0.0,0.003044,0.000000,0.003106
3,0.005848,0.023458,0.012413,0.010286,0.028663,0.005899,0.015161,0.013846,0.010187,0.013895,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.016834,0.000000,0.000000
4,0.002761,0.014942,0.008929,0.008450,0.015153,0.009199,0.011784,0.012234,0.005961,0.008466,...,0.012603,0.004814,0.004796,0.004553,0.009473,0.009670,0.0,0.000000,0.010471,0.004790


In [20]:
# Concatenar el DataFrame tfidf_df con df
df_combined = pd.concat([df[["product_id"]].reset_index(drop=True), tfidf_importantes], axis=1)
df_combined.head()

,product_id,insensitive,skin look,surprised stuff,great warm,told best,happy new,worked completely,revamp,area absorbs,...,add lovely,accordingly,murad really,spf usually,irritation drying,tried otc,allows layer,helped large,line worth,lightweight effective
0,P107306,0.010536,0.009554,0.007561,0.031998,0.007460,0.013716,0.022896,0.010454,0.017137,...,0.000000,0.000000,0.007670,0.000000,0.008947,0.007368,0.0,0.007507,0.000000,0.007661
1,P114902,0.006762,0.012645,0.008969,0.008194,0.008876,0.000000,0.008317,0.007315,0.005093,...,0.003368,0.002700,0.002690,0.002554,0.000000,0.008261,0.0,0.006869,0.004738,0.005638
2,P12045,0.010190,0.009277,0.010387,0.008186,0.010309,0.010279,0.007277,0.010296,0.011717,...,0.000000,0.011777,0.005266,0.004999,0.000000,0.006270,0.0,0.003044,0.000000,0.003106
3,P122651,0.005848,0.023458,0.012413,0.010286,0.028663,0.005899,0.015161,0.013846,0.010187,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.016834,0.000000,0.000000
4,P122661,0.002761,0.014942,0.008929,0.008450,0.015153,0.009199,0.011784,0.012234,0.005961,...,0.012603,0.004814,0.004796,0.004553,0.009473,0.009670,0.0,0.000000,0.010471,0.004790


In [21]:
# product_data = pd.read_csv("data\df_limpio.csv")
product_data = pd.read_csv("data\product_info.csv")

In [22]:
product_data = product_data[["product_id", "brand_name", "rating", "reviews", "price_usd", 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'tertiary_category']]
product_data

,product_id,brand_name,rating,reviews,price_usd,limited_edition,new,online_only,out_of_stock,sephora_exclusive,tertiary_category
0,P473671,19-69,3.6364,11.0,35.0,0,0,1,0,0,Perfume Gift Sets
1,P473668,19-69,4.1538,13.0,195.0,0,0,1,0,0,Perfume
2,P473662,19-69,4.2500,16.0,195.0,0,0,1,0,0,Perfume
3,P473660,19-69,4.4762,21.0,195.0,0,0,1,0,0,Perfume
4,P473658,19-69,3.2308,13.0,195.0,0,0,1,0,0,Perfume
...,...,...,...,...,...,...,...,...,...,...,...
8489,P467659,Yves Saint Laurent,4.4286,7.0,150.0,0,0,0,0,0,Eye Palettes
8490,P500874,Yves Saint Laurent,4.6367,556.0,106.0,0,0,0,0,0,Cologne
8491,P504428,Yves Saint Laurent,5.0000,2.0,134.0,1,1,1,1,0,Perfume Gift Sets
8492,P504448,Yves Saint Laurent,NaN,NaN,167.0,1,1,1,0,0,Cologne Gift Sets


In [23]:
data_completa = df_combined.merge(product_data, on='product_id', how='left')

In [24]:
data_completa = pd.get_dummies(data_completa, columns=['tertiary_category', "brand_name"], dtype=float)

In [25]:
data_completa

,product_id,insensitive,skin look,surprised stuff,great warm,told best,happy new,worked completely,revamp,area absorbs,...,brand_name_bareMinerals,brand_name_belif,brand_name_caliray,brand_name_fresh,brand_name_goop,brand_name_iNNBEAUTY PROJECT,brand_name_iluminage,brand_name_innisfree,brand_name_lilah b.,brand_name_tarte
0,P107306,0.010536,0.009554,0.007561,0.031998,0.007460,0.013716,0.022896,0.010454,0.017137,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,P114902,0.006762,0.012645,0.008969,0.008194,0.008876,0.000000,0.008317,0.007315,0.005093,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,P12045,0.010190,0.009277,0.010387,0.008186,0.010309,0.010279,0.007277,0.010296,0.011717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,P122651,0.005848,0.023458,0.012413,0.010286,0.028663,0.005899,0.015161,0.013846,0.010187,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,P122661,0.002761,0.014942,0.008929,0.008450,0.015153,0.009199,0.011784,0.012234,0.005961,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,P94421,0.010425,0.009670,0.007404,0.006800,0.006736,0.006986,0.007239,0.008267,0.006750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2347,P94812,0.005590,0.015507,0.017526,0.013871,0.015363,0.003330,0.007827,0.015104,0.003397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2348,P9939,0.015194,0.018591,0.018777,0.016482,0.017384,0.012957,0.015763,0.007325,0.014341,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2349,P9940,0.009413,0.011878,0.014080,0.012631,0.011710,0.010900,0.011210,0.010936,0.010294,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df_pca =  data_completa.drop(columns=["product_id"])

In [27]:
from sklearn.preprocessing import StandardScaler
# Aplicar StandardScaler a las columnas numéricas
df_pca = df_pca.astype(float)

# Aplicar StandardScaler a todas las columnas
scaler = StandardScaler()
df_pca = pd.DataFrame(scaler.fit_transform(df_pca), columns=df_pca.columns)

In [28]:
pca = PCA()
pca.fit(df_pca)

# Obtener las componentes principales
X_pca = pca.transform(df_pca)

# Varianza explicada por cada componente principal
explained_variance = pca.explained_variance_ratio_

# Sumar la varianza explicada acumulada
cumulative_explained_variance = explained_variance.cumsum()

In [29]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
plt.title('Varianza Explicada Acumulada por Componentes Principales')
plt.xlabel('Número de Componentes Principales')
plt.ylabel('Varianza Explicada Acumulada')
plt.grid(True)
plt.show()

In [30]:
pca = PCA(n_components=1000)
X_pca_1000 = pca.fit_transform(df_pca)
X_pca_1000

array([[ -3.49876846,   1.54470072, -18.62920328, ...,   0.17291648,
          0.21248435,  -0.67426299],
       [-10.81645869, -14.72652475,  -7.58946032, ...,  -0.44446274,
          0.50518805,   0.20215141],
       [-11.24973551,  -4.47739446,   3.78694954, ...,  -0.05866919,
          0.02675774,  -0.04322421],
       ...,
       [ -5.14616041,  -6.62089066,  -0.1383952 , ...,  -0.85814682,
         -1.19182824,  -1.07584825],
       [-16.02074157,   2.66182394,   5.39827389, ...,   0.07784875,
          0.08269162,   0.42823222],
       [-10.74022026, -15.23285437,   4.50487618, ...,   0.70800598,
          0.98416148,   1.28549019]])

In [31]:
df_pca_1000 = pd.DataFrame(X_pca_1000)

In [32]:
# Concatenar el DataFrame tfidf_df con df
df_pca = pd.concat([df[["product_id"]].reset_index(drop=True), df_pca_1000], axis=1)

In [33]:
df_pca

,product_id,0,1,2,3,4,5,6,7,8,...,990,991,992,993,994,995,996,997,998,999
0,P107306,-3.498768,1.544701,-18.629203,-6.203253,2.543635,-12.993717,9.229074,-2.628540,-0.950772,...,0.130906,-1.479115,-0.185219,0.187342,0.204255,-0.144498,1.367809,0.172916,0.212484,-0.674263
1,P114902,-10.816459,-14.726525,-7.589460,-1.856171,-3.883850,0.696639,-3.799230,-3.923867,-2.159653,...,0.306979,-0.414534,0.107390,-0.363081,0.042755,-0.014755,0.061058,-0.444463,0.505188,0.202151
2,P12045,-11.249736,-4.477394,3.786950,-8.600757,-0.774711,-0.121639,-4.259473,1.903444,-5.568668,...,-0.052789,-0.185072,-0.413884,0.255043,0.622324,0.307794,0.672289,-0.058669,0.026758,-0.043224
3,P122651,-3.390238,-15.544417,6.547343,4.900556,-5.874468,-2.959034,3.982808,1.853453,1.944302,...,0.024786,-0.762893,0.766540,0.151195,-0.514103,0.154546,-1.395851,0.023219,-0.526994,-0.161468
4,P122661,-13.965363,-15.471037,3.882670,1.152329,-2.749192,-4.020367,-1.268166,0.062359,-1.409296,...,-0.007703,0.701839,0.248665,-0.341301,0.355989,-0.495309,0.184838,0.181576,0.565224,-0.372028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,P94421,-13.279747,-3.006598,-5.441225,-2.868487,-2.220278,0.984540,-6.390466,-5.986380,-5.013856,...,-0.169241,-0.215966,0.261958,0.217367,0.195006,-0.025671,0.273515,-0.046342,0.266323,0.179868
2347,P94812,-11.097673,-16.440436,8.510767,2.503445,0.563158,-5.460842,0.691762,1.104771,-3.928707,...,0.690453,-0.179114,-1.015180,0.262133,-0.857750,0.990799,0.411625,-0.863438,-0.426668,0.673138
2348,P9939,-5.146160,-6.620891,-0.138395,2.252234,-4.934860,1.705524,4.063103,4.028719,-4.249524,...,-0.518930,-1.026092,0.049750,0.082681,1.366169,-0.023633,0.010806,-0.858147,-1.191828,-1.075848
2349,P9940,-16.020742,2.661824,5.398274,-9.303194,-3.573102,3.953970,1.409811,-3.542171,-2.549403,...,-0.392524,-0.453174,0.272588,-0.592200,0.448979,0.078488,0.336820,0.077849,0.082692,0.428232


In [34]:
df_pca.to_csv('data/df_pca.csv', index=False)